In [1]:
import re
import nltk
from preprocessing import tokenize_sentences, tokenize_words, pos_tagging, clean_text



     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 6.3 MB/s eta 0:00:02
     ----- ---------------------------------- 1.8/12.8 MB 5.0 MB/s eta 0:00:03
     --------- ------------------------------ 2.9/12.8 MB 4.5 MB/s eta 0:00:03
     ----------- ---------------------------- 3.7/12.8 MB 4.4 MB/s eta 0:00:03
     ------------- -------------------------- 4.2/12.8 MB 4.3 MB/s eta 0:00:03
     ---------------- ----------------------- 5.2/12.8 MB 4.2 MB/s eta 0:00:02
     ------------------ --------------------- 6.0/12.8 MB 4.1 MB/s eta 0:00:02
     --------------------- ------------------ 6.8/12.8 MB 4.1 MB/s eta 0:00:02
     ----------------------- ---------------- 7.6/12.8 MB 4.1 MB/s eta 0:00:02
     -------------------------- ------------- 8.4/12.8 MB 4.1 MB/s eta 0:00:02
     ---------------------------- ----------- 9.2/12.8 MB 4.1 MB/s eta 0:00:01
     ------------------------------- -------- 10.2/12.8 MB 

C:\Users\Mridul\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mridul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mridul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mridul\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mridul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mridul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mridul\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perc

she is dancing beautifully in the park
['She is dancing beautifully in the park!']
Tokens: ['She', 'is', 'dancing', 'beautifully', 'in', 'the', 'park', '!']
Filtered Tokens: ['dancing', 'beautifully', 'park', '!']
Tagged Tokens: [('She', 'PRP'), ('is', 'VBZ'), ('dancing', 'VBG'), ('beautifully', 'RB'), ('in', 'IN'), ('the', 'DT'), ('park', 'NN'), ('!', '.')]


In [2]:
def is_imperative(sentence):
    """
    Check if a sentence is likely an imperative sentence.
    
    Heuristic:
    - If the first meaningful word is a base form verb (VB), it's likely imperative.
    - Ignores politeness words like 'please' and 'kindly'.
    
    Parameters:
    sentence (str): The input sentence.
    
    Returns:
    bool: True if the sentence is imperative, False otherwise.
    """
    polite_words = {"please", "kindly"}

    tokens = tokenize_words(sentence)
    if not tokens:
        return False

    # Convert tokens to lowercase and filter out polite words
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in polite_words]

    if not filtered_tokens:  # If all tokens were polite words, return False
        return False

    tagged = pos_tagging(filtered_tokens)
    
    # Check if the first meaningful word is a base form verb (VB)
    return tagged[0][1] == 'VB'


In [3]:
import re

def contains_task_indicators(sentence):
    """
    Check if the sentence contains phrases that indicate a task.
    
    Uses a set of predefined indicator phrases.
    
    Parameters:
    sentence (str): The input sentence.
    
    Returns:
    bool: True if a task-related phrase is found, False otherwise.
    """
    indicators = [r"\bhas to\b", r"\bshould\b", r"\bmust\b", r"\bneeds to\b", r"\bought to\b"]
    
    sentence_lower = sentence.lower()
    
    return any(re.search(pattern, sentence_lower) for pattern in indicators)


In [4]:
import re

def extract_person(sentence):
    pattern = r'((?:[A-Za-z][a-z]*\s*)+)(?=\s+(has to|should|must|needs to|ought to))'
    match = re.search(pattern, sentence)
    
    if match:
        extracted = match.group(1).strip()
        print("Extracted person:", extracted)
        return extracted
    
    print("No person match found in:", sentence)
    return None


In [4]:
def extract_deadline(sentence):
    """
    Attempt to extract deadline information from the sentence.
    Heuristic: Look for phrases starting with 'by', 'before', or 'at' followed by time expressions or day indicators.
    
    Examples:
      "John should clean the room by 5 pm today"      -> Extracts "5 pm today"
      "Michael needs to call the client before noon today" -> Extracts "noon today"
      "Priya needs to submit the report by tomorrow"   -> Extracts "tomorrow"
      "Send the email by midnight."                   -> Extracts "midnight"
      "Prepare for the meeting by next Monday."       -> Extracts "next Monday"
      "Check the documents by early morning."         -> Extracts "early morning"
      "Mark has to schedule a meeting at 9 am tomorrow."-> Extracts "9 am tomorrow"
    """
    deadline_pattern = r'\b(?:by|before|at)\s+((?:(?:\d{1,2}\s*(?:am|pm))|noon|midday|midnight)(?:\s+\w+)?|tomorrow|today|tonight|next\s+\w+day|\w+day|end of day|early morning|late night)(?=\b|[\s\.,]|$)'
    match = re.search(deadline_pattern, sentence, re.IGNORECASE)
    if match:
        return match.group(1)
    return None

In [5]:
from datetime import datetime, timedelta
import re

from datetime import datetime, timedelta
import re

def remove_ordinal_suffix(date_str):
    """Remove ordinal suffixes ('st', 'nd', 'rd', 'th') from date strings."""
    return re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)

def convert_to_datetime(deadline):
    """
    Convert an extracted deadline string into a datetime object, if possible.
    
    - For relative dates like "tomorrow", "today", it uses the current date.
    - For time-only strings like "5 pm", it attaches today's date.
    - For absolute dates like "March 10th", it assumes the current year.
    """
    today = datetime.today()
    deadline_str = deadline.strip()  # preserve original case for parsing
    deadline_lower = deadline_str.lower()  # for comparisons

    # Handle relative dates
    if "tomorrow" in deadline_lower:
        return today + timedelta(days=1)
    elif "today" in deadline_lower or "tonight" in deadline_lower:
        return today

    # Handle time-only deadlines like "5 pm"
    time_match = re.match(r'(\d{1,2})\s*(am|pm)', deadline_str, re.IGNORECASE)
    if time_match:
        hour = int(time_match.group(1))
        period = time_match.group(2).lower()
        if period == "pm" and hour != 12:
            hour += 12
        elif period == "am" and hour == 12:
            hour = 0
        return today.replace(hour=hour, minute=0, second=0, microsecond=0)

    # Handle absolute dates like "March 10th" or "March 15th"
    # Remove ordinal suffixes, but keep the original casing for the month.
    deadline_clean = remove_ordinal_suffix(deadline_str).strip()
    try:
        # Expecting format like "March 10"
        parsed_date = datetime.strptime(deadline_clean, "%B %d")
        # Assume the current year for the deadline
        return parsed_date.replace(year=today.year)
    except ValueError:
        return None

In [6]:
def extract_deadlines(sentence):
    """
    Extract deadline-related expressions from a sentence and convert them to datetime.
    """
    deadline_pattern = (
        r'\b(?:by|before|at|within|in|after)\s+('
        r'(?:\d{1,2}:\d{2}\s*(?:am|pm)?)|'  # Matches "5:30 pm"
        r'(?:\d{1,2}\s*(?:am|pm))|'  # Matches "5 pm"
        r'(?:noon|midday|midnight)|'  # Matches "noon"
        r'(?:tomorrow|today|tonight)|'  # Matches "tomorrow"
        r'(?:next\s+\w+day|\w+day)|'  # Matches "next Monday"
        r'(?:end of (?:day|week))|'  # Matches "end of day"
        r'(?:early morning|late night)|'  # Matches "early morning"
        r'(?:this\s+\w+)|'  # Matches "this evening"
        r'(?:\d{1,2}(?:st|nd|rd|th)?\s+\w+)|'  # Matches "15th March"
        r'(?:\w+\s+\d{1,2}(?:st|nd|rd|th)?)|'  # Matches "March 15th"
        r'(?:\d{4}-\d{2}-\d{2})|'  # Matches "2025-02-20"
        r'(?:\d+\s+(?:hours?|minutes?|days?|weeks?|months?|years?))'  # Matches "in 3 days"
        r')'
    )

    matches = re.findall(deadline_pattern, sentence, re.IGNORECASE)
    
    extracted_strings = [match.strip() for match in matches if match]
    extracted_dates = [convert_to_datetime(deadline) for deadline in extracted_strings]

    return extracted_strings, extracted_dates


# ✅ Example Usage
sentence = "Submit by March 10th or latest by March 15th at 5 pm."
extracted, dates = extract_deadlines(sentence)

print("Extracted Deadlines:", extracted)
print("Converted Dates:", dates)

Extracted Deadlines: ['March 10th', 'March 15th', '5 pm']
Converted Dates: [datetime.datetime(2025, 3, 10, 0, 0), datetime.datetime(2025, 3, 15, 0, 0), datetime.datetime(2025, 2, 16, 17, 0)]


In [11]:
def extract_tasks(raw_text):
    """
    Extract tasks from raw text by identifying imperative sentences, 
    task indicator phrases, and deadlines.
    
    Returns:
        A list of dictionaries with keys: 'task', 'person', 'deadline'
    """
    sentences = tokenize_sentences(raw_text)
    tasks = []
    
    for sentence in sentences:
        cleaned_sentence = clean_text(sentence)
        deadline = extract_deadline(cleaned_sentence)  # Store deadline to avoid duplicate calls

        if is_imperative(sentence) or contains_task_indicators(cleaned_sentence) or deadline:
            person = extract_person(sentence) or "Unassigned"  # Default if no person is found
            tasks.append({
                'task': sentence,
                'person': person,
                'deadline': deadline
            })
        else:
            tasks.append('This is not task')

    # Return the extracted tasks; remove or adjust the static default task as needed.
    return tasks



In [12]:
extract_tasks("John is a good boy")

['This is not task']

In [10]:
if __name__ == "__main__":
    sample_text = (
        "John is playing games since yesterday. "
        "Adam will go to church tomorrow. "
        "Sarah should update the system by 4 pm. \n\n"
        "Submit the report by 5 pm today. \n"
        "Mark has to schedule a meeting at 9 am tomorrow. \n"
        "Prepare for the meeting by next Monday. \n"
        "The weather is nice today."
    )
    
    extracted_tasks = extract_tasks(sample_text)
    for idx, task in enumerate(extracted_tasks, 1):
        print(f"Task {idx}: {task['task']}")
        print(f"    Person: {task['person'] if task['person'] else 'Not Specified'}")
        print(f"    Deadline: {task['deadline'] if task['deadline'] else 'Not Specified'}\n")

Extracted person: Sarah
No person match found in: Submit the report by 5 pm today.
Extracted person: Mark
No person match found in: Prepare for the meeting by next Monday.
Task 1: Sarah should update the system by 4 pm.
    Person: Sarah
    Deadline: 4 pm

Task 2: Submit the report by 5 pm today.
    Person: Unassigned
    Deadline: 5 pm today

Task 3: Mark has to schedule a meeting at 9 am tomorrow.
    Person: Mark
    Deadline: 9 am tomorrow

Task 4: Prepare for the meeting by next Monday.
    Person: Unassigned
    Deadline: next monday

